In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install catboost
# !pip install textfeatures

In [ ]:
import numpy as np
import pandas as pd
import textfeatures as FE
from catboost import CatBoostClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def softmax(x):
    
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

In [ ]:
train_db = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/Data/Feedback Prize - Predicting Effective Arguments/TRAIN_DB.csv')#[['discourse_id', 'discourse_text']]
test_db = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/Data/Feedback Prize - Predicting Effective Arguments/TEST_DB.csv')#[['discourse_id', 'discourse_text']]

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/Data/Feedback Prize - Predicting Effective Arguments/TRAIN_CATBOOST.csv')
test = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/Data/Feedback Prize - Predicting Effective Arguments/TEST_CATBOOST.csv')

In [ ]:
features_col = ['discourse_id', 'dis_word_count',
                'dis_char_count', 'dis_avg_word_length', 'dis_stopwords_count',
                'dis_paragraph', 'essay_word_count', 'essay_char_count',
                'essay_avg_word_length', 'essay_stopwords_count', 'essay_paragraph']

In [ ]:
DBf0 = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/experiment/DB4large_FOLD0/prediction[0].csv')[['discourse_id', 'Ineffective', 'Adequate', 'Effective']]
DBf1 = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/experiment/DB4large__FOLD1/prediction[1].csv')[['discourse_id', 'Ineffective', 'Adequate', 'Effective']]
DBf2 = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/experiment/DB4large__FOLD2/prediction[2].csv')[['discourse_id', 'Ineffective', 'Adequate', 'Effective']]
DBf3 = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/experiment/DB4large__FOLD3/prediction[3].csv')[['discourse_id', 'Ineffective', 'Adequate', 'Effective']]
DBf4 = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/experiment/DB4large__FOLD4/prediction[4].csv')[['discourse_id', 'Ineffective', 'Adequate', 'Effective']]

In [ ]:
preds_db = pd.concat([DBf0, DBf1, DBf2, DBf3, DBf4])

In [ ]:
train = train.merge(preds_db, left_on='discourse_id', right_on='discourse_id')

In [ ]:
train[['Ineffective', 'Adequate', 'Effective']] = train[['Ineffective', 'Adequate', 'Effective']].apply(softmax, axis=1)

In [ ]:
DB_test_preds = pd.read_csv('/content/drive/MyDrive/FeedBack_Prize/Data/Feedback Prize - Predicting Effective Arguments/predicted_test_by_v5DBlS42.csv')

In [ ]:
test = test.merge(DB_test_preds, left_on='discourse_id', right_on='discourse_id')

In [ ]:
train = train.merge(train_db[['discourse_id', 'essay_id', 'discourse_text', 'essay']], left_on=['discourse_id', 'essay_id'], right_on=['discourse_id', 'essay_id'])
test = test.merge(test_db[['discourse_id', 'essay_id', 'discourse_text', 'essay']], left_on=['discourse_id', 'essay_id'], right_on=['discourse_id', 'essay_id'])

In [ ]:
def create_emb_essay(df):
    essay_emb = {}
    _essayList = df.essay_id.unique()
    for essay in _essayList:
        emb = df[df.essay_id == essay].sort_values(by='pos')[['Ineffective',
                                                              'Adequate', 'Effective']].to_numpy().flatten()
        g = np.zeros(72)
        g[:emb.size] = emb
        essay_emb[essay] = g
    df = df.merge(pd.Series(essay_emb).reset_index(), left_on='essay_id', right_on='index')
    df.drop('index', axis=1, inplace=True)
    df.rename(columns={0:'embeddings_essay'}, inplace=True)
    return df


def text_features_eng(df):
    col = ['Ineffective', 'Adequate', 'Effective']
    FE.word_count(df, 'discourse_text', 'dis_word_count')
    FE.char_count(df, 'discourse_text', 'dis_char_count')
    FE.avg_word_length(df, 'discourse_text', 'dis_avg_word_length')
    FE.stopwords_count(df, 'discourse_text', 'dis_stopwords_count')
    df['dis_paragraph'] = df.discourse_text.map(lambda i: i.count('\n'))
    FE.word_count(df, 'essay', 'essay_word_count')
    FE.char_count(df, 'essay', 'essay_char_count')
    FE.avg_word_length(df, 'essay', 'essay_avg_word_length')
    FE.stopwords_count(df, 'essay', 'essay_stopwords_count')
    df['essay_paragraph'] = df.essay.map(lambda i: i.count('\n'))
    essay_sum_scores = df.groupby('essay_id')[col].sum().reset_index()
    df = df.merge(essay_sum_scores, left_on='essay_id', right_on='essay_id', suffixes=('', '_'))
    #df[['Ineffective_', 'Adequate_', 'Effective_']] = (df[['Ineffective_', 'Adequate_', 'Effective_']].to_numpy())/df.size_essay.to_numpy().reshape(len(df), 1)
    #df = create_emb_essay(df)

    return df

In [ ]:
train = text_features_eng(train)
test = text_features_eng(test)

In [ ]:
select = ['code_topic',
          'code_dis_type',
          'start_pos',
          'end_pos',
          'len_text',
          'len_essay',
          'size_essay',
          'pos',
          'dis_word_count',
          #'dis_char_count',
          'dis_avg_word_length',
          #'dis_paragraph',
          # 'essay_word_count',
          # 'essay_char_count',
          #'essay_avg_word_length',
          # 'essay_stopwords_count',
          #'essay_paragraph',
          #'embeddings_essay',
          'Ineffective_',
          'Adequate_',
          'Effective_',
          'Ineffective',
          'Adequate',
          'Effective']

In [ ]:
# from sklearn import preprocessing
# scaler = preprocessing.StandardScaler().fit(train[select[2:]])
#train[select[2:]] = scaler.transform(train[select[2:]])
train_set = train[select]
train_lab = train['label']

#test[select[2:]] = scaler.transform(test[select[2:]])
test_set = test[select]
test_lab = test['label']


In [ ]:
params1 = {
    #'use_best_model': True,
    #'class_weights': [1.5, 1.0, 1.25],
    #'boost_from_average': True,
    'iterations': 2.5e4,
    'learning_rate': 1.5e-3,
    'train_dir': 'MultiClass',
    'bagging_temperature': 1,
    'leaf_estimation_method' : 'Newton',
    'boosting_type' : 'Ordered', 
    'loss_function': 'MultiClass',
    'eval_metric': 'MultiClass',
    'cat_features': ['code_dis_type', 'code_topic'],
    #'embedding_features': ['embeddings_essay'],
    'l2_leaf_reg': 1.1,
    'random_seed': 1602,
    'max_depth': 7,
    #'text_features': ['discourse_text'],
    #'task_type': 'GPU',
    'early_stopping_rounds': 1000}
param_grid = {'max_depth': [6, 7],
              'random_seed': [42, 1602],
              'l2_leaf_reg': [1.1, 1.2, 1.3, 1.4],
              'learning_rate': [3.5e-3, 2.5e-3, 1.5e-3]}

In [ ]:
model = CatBoostClassifier(**params1)

In [ ]:
# grid_search_result = model.grid_search(param_grid, 
#                                        X=train_set,
#                                        y=train_lab)

In [ ]:
# grid_search_result['params']

In [ ]:
model.fit(train_set, train_lab, eval_set=(test_set, test_lab))

Streaming output truncated to the last 5000 lines.
1607:	learn: 0.5978248	test: 0.6355338	best: 0.6355338 (1607)	total: 7m 40s	remaining: 1h 51m 40s
1608:	learn: 0.5977980	test: 0.6355161	best: 0.6355161 (1608)	total: 7m 40s	remaining: 1h 51m 40s
1609:	learn: 0.5977863	test: 0.6355067	best: 0.6355067 (1609)	total: 7m 41s	remaining: 1h 51m 40s
1610:	learn: 0.5977773	test: 0.6354931	best: 0.6354931 (1610)	total: 7m 41s	remaining: 1h 51m 38s
1611:	learn: 0.5977700	test: 0.6354848	best: 0.6354848 (1611)	total: 7m 41s	remaining: 1h 51m 38s
1612:	learn: 0.5977523	test: 0.6354764	best: 0.6354764 (1612)	total: 7m 42s	remaining: 1h 51m 38s
1613:	learn: 0.5977432	test: 0.6354709	best: 0.6354709 (1613)	total: 7m 42s	remaining: 1h 51m 37s
1614:	learn: 0.5977331	test: 0.6354769	best: 0.6354709 (1613)	total: 7m 42s	remaining: 1h 51m 36s
1615:	learn: 0.5977248	test: 0.6354778	best: 0.6354709 (1613)	total: 7m 42s	remaining: 1h 51m 36s
1616:	learn: 0.5977032	test: 0.6354649	best: 0.6354649 (1616)	total

In [ ]:
#g = {k:v for k, v in zip(select, model.feature_importances_)}

In [ ]:
#model.feature_importances_

In [ ]:
#sorted(g, key=lambda r: g[r])

In [ ]:
from sklearn.metrics import log_loss
label = np.zeros((len(test_lab), 3))
label[np.arange(len(test_lab)), test_lab.tolist()] = 1.0
q = model.predict_proba(test_set)
debert_log_loss = log_loss(label, test[['Ineffective', 'Adequate', 'Effective']].values)
#robert_log_loss = log_loss(label, test[['Ineffective_y', 'Adequate_y', 'Effective_y']].values)
catboost_log_loss = log_loss(label, q)
print(f'log_loss_DeBert {debert_log_loss}')
#print(f'log_loss_RoBert {robert_log_loss}')
print(f'log_loss_catboost {catboost_log_loss}')
print(f'impruved {debert_log_loss - catboost_log_loss}')

log_loss_DeBert 0.644679546975412
log_loss_catboost 0.6264887228010252
impruved 0.018190824174386777


In [ ]:
model.save_model("/content/drive/MyDrive/FeedBack_Prize/cat_boost_16")

In [ ]:
{'select_15': ['code_topic', 'code_dis_type', 'start_pos', 'end_pos', 'len_text',
               'len_essay', 'size_essay', 'pos', 'dis_word_count', 'dis_avg_word_length',
               'dis_stopwords_count', 'Ineffective', 'Adequate', 'Effective']}